In [1]:
import numpy as np
import math
from bounding_box_utils.bounding_box_utils import iou, convert_coordinates
from ssd_encoder_decoder.matching_utils import match_bipartite_greedy, match_multi
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [2]:
sess = tf.Session()

K.set_session(sess)

In [3]:
gt = np.load("outputs/predder.npy")

In [4]:
gt1 = gt[1]['predictions_1']
gt2 = gt[1]['predictions_2']
gt1_proj = gt[1]['predictions_1_proj_tot']
gt2_proj = gt[1]['predictions_2_proj_tot']

In [5]:
pred_1 = np.load("outputs/predictions_1_14.npy")
pred_1_proj = np.load("outputs/predictions_1_proj_14.npy")
pred_2 = np.load("outputs/predictions_2_14.npy")
pred_2_proj = np.load("outputs/predictions_2_proj_14.npy")

labels:  (5, 4)
y_encoded:  (17292, 4)
similarities:  (5, 17292)
bipartite_matches:  (5,)
similarities after bipartite:  (5, 17292)
matches:  (53,)  -  (53,)
similarities after matches:  (5, 17292)

In [56]:
pred_1 = pred_1_proj[:,:,:18]
gt_1 = gt1_proj[:,:,:18]
gt_2 = gt2_proj[:,:,18:]

In [57]:
filterer = np.where(pred_1[:,:,0]>0.5)
filterer2 = np.where(gt_1[:,:,-1]!=99)

In [58]:
# src = src[filterer]
pred_1 = pred_1[0,:,:]
gt_1 = gt_1[filterer2]
gt_2 = gt_2[filterer2]

In [59]:
# src = src[0,:,:]
print(pred_1.shape)
print(gt_1.shape)
print(gt_2.shape)

(17292, 18)
(4, 18)
(4, 18)


In [ ]:
pred_2 = pred_1_proj[:,:,18:]

In [ ]:
iou_out = tf.py_func(iou, [tf.convert_to_tensor(gt_1[:,-16:-12]), tf.convert_to_tensor(pred_1[:,-16:-12])], tf.float64)

In [ ]:
bipartite_matches = tf.py_func(match_bipartite_greedy, [iou_out], tf.int64)
print(K.eval(bipartite_matches).shape)

In [ ]:
def finder(tensor, matches):
    return tensor[:,matches,:]
    

In [ ]:
t = tf.py_func(finder, [tf.convert_to_tensor(pred_2),bipartite_matches], tf.float32)

In [ ]:
all_elems_equal = tf.reduce_all(tf.equal(tf.shape(t)[1], tf.shape(t)[1]))

In [ ]:
K.eval(all_elems_equal)

In [71]:
y_true_1 = gt1_proj[:,:,:18]
y_pred_1 = pred_1_proj[:,:,18:]
y_true_2 = gt1_proj[:,:,18:]
y_pred_2 = pred_1_proj[:,:,18:]

In [72]:
batch_size = tf.shape(y_pred_1)[0]

In [86]:
batch_size = 1
def finder(tensor, matches):
    return tensor[:,matches,:]


def matcher(y_true_1,y_pred_1,y_pred_2, bsz):
    out = []
    if bsz > 0:
        for i in range(bsz):
            filterer = np.where(y_true_1[i,:,-1]!=99)
            y_true_1 = y_true_1[i,filterer,:]
            iou_out = tf.py_func(iou, [tf.convert_to_tensor(y_true_1[i,:,-16:-12]), tf.convert_to_tensor(y_pred_1[i,:,-16:-12])], tf.float64)
            bipartite_matches = tf.py_func(match_bipartite_greedy, [iou_out], tf.int64)
            out = tf.py_func(finder, [tf.convert_to_tensor(y_pred_2),bipartite_matches], tf.float32)
            #append or add here
    else:
        for i in range(bsz):
            filterer = np.where(y_true_1[i,:,-1]!=99)
            y_true_1 = y_true_1[i,filterer,:]
            iou_out = tf.py_func(iou, [tf.convert_to_tensor(y_true_1[i,:,-16:-12]), tf.convert_to_tensor(y_pred_1[i,:,-16:-12])], tf.float64)
            bipartite_matches = tf.py_func(match_bipartite_greedy, [iou_out], tf.int64)
            out = tf.py_func(finder, [tf.convert_to_tensor(y_pred_2),bipartite_matches], tf.float32)
            np.concatenate()

    return out


In [87]:
res = matcher(y_true_1,y_pred_1,y_pred_2, batch_size)

In [88]:
K.eval(res)

array([[[9.0946209e-01, 9.0537898e-02, 2.0679751e-04, 5.1759202e-05,
         0.0000000e+00, 2.6740139e-05, 6.6666668e-03, 1.3333334e-02,
         5.0000001e-02, 1.0000000e-01, 1.0000000e-01, 1.0000000e-01,
         2.0000000e-01, 2.0000000e-01, 1.0000000e+00, 1.0000000e+00,
         1.0000000e+00, 1.0000000e+00],
        [9.0946209e-01, 9.0537898e-02, 2.0679751e-04, 5.1759202e-05,
         0.0000000e+00, 2.6740139e-05, 6.6666668e-03, 1.3333334e-02,
         5.0000001e-02, 1.0000000e-01, 1.0000000e-01, 1.0000000e-01,
         2.0000000e-01, 2.0000000e-01, 1.0000000e+00, 1.0000000e+00,
         1.0000000e+00, 1.0000000e+00],
        [9.0946209e-01, 9.0537898e-02, 2.0679751e-04, 5.1759202e-05,
         0.0000000e+00, 2.6740139e-05, 6.6666668e-03, 1.3333334e-02,
         5.0000001e-02, 1.0000000e-01, 1.0000000e-01, 1.0000000e-01,
         2.0000000e-01, 2.0000000e-01, 1.0000000e+00, 1.0000000e+00,
         1.0000000e+00, 1.0000000e+00],
        [9.0946209e-01, 9.0537898e-02, 2.0679751e-04

In [48]:
y_true_1.shape

(1, 17292, 18)

In [73]:
filterer = np.where(y_true_1[0,:,-1]!=99)

In [74]:
filterer

(array([ 4845, 16112, 16129, 17272]),)

In [75]:
y_true_1 = y_true_1[0,filterer,:]
y_true_2 = y_true_2[0,filterer,:]



In [76]:
y_true_1.shape

(1, 4, 18)

In [77]:
iou_out = tf.py_func(iou, [tf.convert_to_tensor(y_true_1[0,:,-16:-12]), tf.convert_to_tensor(y_pred_1[0,:,-16:-12])], tf.float64)
print(K.eval(iou_out))
print(K.eval(iou_out).shape)

[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [-0. -0. -0. ... -0. -0. -0.]
 [-0. -0. -0. ... -0. -0. -0.]]
(4, 17292)


In [ ]:
K.eval(out)

In [78]:
bipartite_matches = tf.py_func(match_bipartite_greedy, [iou_out], tf.int64)


In [80]:
K.eval(bipartite_matches)

array([0, 0, 0, 0])

In [ ]:
K